## Toggle Flip-Flop

In this example we create a toggle flip-flop (TFF) from a d-flip-flop and an xor gate. In `Magma`, sequential logic circuits can be constructed by composing combinational logic with register primitives, such as the ice40 DFF.

In [1]:
import magma as m
m.set_mantle_target("ice40")

from mantle import DFF

import lattice ice40
import lattice mantle40


As before, we can use a native Python function to organize the definition of our TFF into a reusable component.

In [2]:
def tff():
    # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
    ff = DFF()

    # compute the next state as the not of the old state ff.O
    ff( ~ff.O )

    return ff.O

Then we simply call this function inside our definition of the IceStick `main`.

In [3]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J3[0].rename('J3').output().on()

main = icestick.DefineMain()
m.wire( tff(), main.J3 )
m.EndDefine()

We'll compile and build our program using the standard flow.

In [4]:
m.compile("build/tff", main, vendor="lattice")

compiling main


In [5]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif tff.blif' tff.v
arachne-pnr -q -d 1k -o tff.txt -p tff.pcf tff.blif 
icepack tff.txt tff.bin
iceprog tff.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


Let's inspect the generated verilog.

In [6]:
%cat build/tff.v

module main (output  J3, input  CLKIN);
wire  inst0_Q;
wire  inst1_O;
SB_DFF inst0 (.C(CLKIN), .D(inst1_O), .Q(inst0_Q));
SB_LUT4 #(.LUT_INIT(16'h5555)) inst1 (.I0(inst0_Q), .I1(1'b0), .I2(1'b0), .I3(1'b0), .O(inst1_O));
assign J3 = inst0_Q;
endmodule



We can verify our implementation is function correctly by using a logic analyzer.

In [7]:
%cat build/tff.pcf

set_io J3 62
set_io CLKIN 21


![](images/tff.png)